In [ ]:
SEED = 0

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
%cd /content/gdrive/My Drive/Kaggle

Mounted at /content/gdrive
/content/gdrive/My Drive/Kaggle


In [ ]:
#Loading data
import pandas as pd
import numpy as np

data = pd.read_csv("/content/gdrive/My Drive/Kaggle/creditcard.csv", header=0)
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
#Handle Duplicated values
print("Duplicates", data.duplicated().sum())
data = data.drop_duplicates()
print("Rows {}, Columns {}".format(data.shape[0], data.shape[1]))

Duplicates 1081
Rows 283726, Columns 31


In [ ]:
data['Class'] = data['Class'].replace({'0': 0, '1': 1})
data['Class'].unique()
data['Class'].value_counts()

0    283253
1       473
Name: Class, dtype: int64

In [ ]:
labels = data['Class'] 
#data = data[data.columns[:-1]]
labels.value_counts()

0    283253
1       473
Name: Class, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( data, labels, test_size=0.3, random_state=SEED )

In [ ]:
#One class SVM
import pandas as pd
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# Separate the minority class samples from the majority class samples in the training set

X_train['Class'] = y_train
X_train_minority = X_train[X_train['Class'] == 1]
print(X_train_minority.shape, X_train.shape)

# Define the One-Class SVM model and fit it to the minority class samples
ocsvm = OneClassSVM(kernel='rbf', nu=0.05, gamma='scale')
ocsvm.fit(X_train_minority)

# Evaluate the One-Class SVM model on the testing set
y_pred = ocsvm.predict(X_test)

# Convert the predicted labels from -1 and 1 to 0 and 1
y_pred[y_pred == -1] = 0
y_pred[y_pred == 1] = 1

# Calculate and print the performance metrics
print("Accuracy Score: {:.2f}%".format(accuracy_score(y_test, y_pred) * 100))
print("F1 Score: {:.2f}%".format(f1_score(y_test, y_pred) * 100))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification report\n", classification_report(y_test, y_pred))


(344, 31) (198608, 31)
Accuracy Score: 9.47%
F1 Score: 0.31%
Confusion Matrix:
 [[ 7943 77046]
 [   10   119]]
Classification report
               precision    recall  f1-score   support

           0       1.00      0.09      0.17     84989
           1       0.00      0.92      0.00       129

    accuracy                           0.09     85118
   macro avg       0.50      0.51      0.09     85118
weighted avg       1.00      0.09      0.17     85118



            Time         V1         V2         V3         V4         V5  \
42696    41203.0  -8.426814   6.241659  -9.946470   8.199614  -8.213093   
262826  160665.0  -0.417340   4.700055  -7.521767   7.671884   0.260821   
157585  110087.0   1.934946   0.650678  -0.286957   3.987828   0.316052   
15539    26931.0 -22.561699  13.208904 -24.643819   6.232532 -16.905611   
42473    41116.0  -3.600544   4.519047  -6.340884   6.214767  -5.829558   
...          ...        ...        ...        ...        ...        ...   
239499  150138.0  -2.150855   2.187917  -3.430516   0.119476  -0.173210   
123301   76876.0  -1.298359   1.079671  -0.180678   1.287839   1.858273   
70141    53727.0  -1.649279   1.263974  -1.050826   2.237991  -2.527889   
150654   93834.0  -3.765680   5.890735 -10.202268  10.259036  -5.611448   
221041  142409.0  -1.172183   1.661713  -3.049637   2.555058   3.669035   

              V6         V7         V8        V9  ...       V21       V22  \
42696  -2.522046 -11.6

In [ ]:
#Isolation forest
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split

iforest = IsolationForest(
    n_estimators=100,
    max_samples='auto',
    contamination='auto',
    random_state=SEED
)

# Train the Isolation Forest model on the training set
iforest.fit(X_train)

# Predict the anomaly scores for the testing set
y_scores = iforest.decision_function(X_test)

# Convert the anomaly scores to binary labels
y_pred = [1 if s < 0 else 0 for s in y_scores]

# Calculate and print the performance metrics
print("Accuracy Score: {:.2f}%".format(accuracy_score(y_test, y_pred) * 100))
print("F1 Score: {:.2f}%".format(f1_score(y_test, y_pred) * 100))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy Score: 96.42%
F1 Score: 6.45%
Confusion Matrix:
 [[81967  3022]
 [   24   105]]


In [ ]:
#LOF

import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.neighbors import LocalOutlierFactor

# Define the Local Outlier Factor detector
lof = LocalOutlierFactor(n_neighbors=20, contamination='auto')

# Fit the LOF detector on the training set
lof.fit(X_train)

# Use LOF to identify outliers in the training set
lof_scores_train = lof.negative_outlier_factor_
lof_threshold_train = -2.0
outliers_train = X_train[lof_scores_train < lof_threshold_train]

# Remove the identified outliers from the training set
X_train_clean = X_train[lof_scores_train >= lof_threshold_train]
y_train_clean = y_train[lof_scores_train >= lof_threshold_train]

# Define the Gradient Boosting classifier
gb_clf = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=SEED
)

# Train the Gradient Boosting classifier on the cleaned training set
gb_clf.fit(X_train_clean, y_train_clean)

# Evaluate the Gradient Boosting classifier on the original testing set
y_pred = gb_clf.predict(X_test)

# Calculate and print the performance metrics
print("Accuracy Score: {:.2f}%".format(accuracy_score(y_test, y_pred) * 100))
print("F1 Score: {:.2f}%".format(f1_score(y_test, y_pred) * 100))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification report:\n", classification_report(y_test, y_pred))

Accuracy Score: 100.00%
F1 Score: 100.00%
Confusion Matrix:
 [[84989     0]
 [    0   129]]
Classification report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     84989
           1       1.00      1.00      1.00       129

    accuracy                           1.00     85118
   macro avg       1.00      1.00      1.00     85118
weighted avg       1.00      1.00      1.00     85118

